In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import datetime

1) Log file is loaded

In [3]:
file = open("log.txt", "r") 
lines = file.readlines()
for l in range(3): print(lines[l][:-1])

156.15.13.173 [01/Jan/2018:11:13:49 +0100] "GET AddProduct.html HTTP/1.0" 200 2356
176.247.135.49 [01/Jan/2018:11:18:07 +0100] "GET Shop.html HTTP/1.0" 200 7466
176.247.135.49 [01/Jan/2018:11:19:05 +0100] "GET AddProduct.html HTTP/1.0" 200 2356


2) Cass "Request" parses and keeps data of a single requests. Complete the TODO, where a given "line" (string) should be parsed in order to dervie the ip of the host making the request, time of the request (in seconds), and target page (name).

In [4]:
class Request:
    def __init__(self, line):
        # TODO DO PARSING HERE
        
        self.ip = line.split(' ')[0]
        # convert hours + minutes + seconds -> total number of seconds
        str = line.split(' ')[1].split(':')
        self.time = int(str[-1]) + (int(str[-2]) * 60) + (int(str[-3]) * 3600)
        self.page = line.split(' ')[4]
        
    def __repr__(self):
        return self.ip + "     " +  str(datetime.timedelta(seconds=self.time)) + " -> " + self.page


3) Each line of log.txt is parsed and the Request objects are added to the list

In [5]:
reqs = [Request(line) for line in lines]
for l in range(10): print(reqs[l]) 

156.15.13.173     11:13:49 -> AddProduct.html
176.247.135.49     11:18:07 -> Shop.html
176.247.135.49     11:19:05 -> AddProduct.html
176.247.135.49     11:29:05 -> Summary.html
69.84.30.199     11:43:59 -> Shop.html
69.84.30.199     11:45:16 -> AddProduct.html
57.116.45.120     11:46:45 -> AddProduct.html
69.84.30.199     11:50:36 -> Summary.html
69.84.30.199     11:52:38 -> Payment.html
26.19.183.202     11:53:11 -> AddProduct.html


# Exercise

Your task is to analyze the log file. You should indentify the users and sessions. Then, compute the following: <br>
3.1) the number of sessions (should be 1000 :) ), <br>
3.2) the number of users, <br>
3.3) the average number of sessions per one user <br>
3.4) use the distribution plot to visualize the sessions' lengths (minutes);  <br>
3.5) use the distribution plot to visualize the sessions' lengths (the number of requests) <br>
3.6) use the distribution plot to visualize sessions' starting times (hours). <br>
3.7) which web page is the most common entry point?; for each page plot the number of times it has been an entry page,  <br>
3.8) which web page is the most common departure point?; for each page plot the number of times the user stopped a session while being on this page, <br>
3.9) for each page plot the average time the user stayed on this page <br>
3.10) print the fist 10 sessions (paths). <br>

You may find the following classes helpful and data.

In [ ]:
class Session:
    def __init__(self, start, req):
        self.startTime = start
        self.stopTime = start
        self.requests = []
        self.requests.append(req)
        
    def __repr__(self):
        print("SESSION")
        print(self.startTime)
        print(self.stopTime)
        STR = ""
        for r in self.requests:
            print(r)
            STR += ("  " + r.page + "    " + str(datetime.timedelta(seconds=r.time)) + "\n")
        return STR

class Computer:
    def __init__(self, ip, session):
        self.ip = ip
        self.sessions = []
        self.sessions.append(session)
    def __repr__(self):
        return self.ip

PAGES = ["Shop.html", "AddProduct.html", "Summary.html", "Payment.html","About.html", "Contact.html"]

LINKS = [[1, 4],
         [2,4],
         [3],
         [],
         [1,5],
         []]

file = open("log.txt", "r") 
lines = file.readlines()
linesCount = 0

for line in lines:
    linesCount += 1
print("Liczba logów: ", linesCount)

th1 = 800 # treshold on max time on single page
th2 = 2400 # treshold on max time in one session

computers = []
IPs = set()
for i in range(len(reqs)):
    ip = reqs[i].ip
    time = reqs[i].time
    if not ip in IPs:
        IPs.add(ip)
        computers.append(Computer(ip, Session(time, reqs[i])))
    else:
        comps = [c for c in computers if c.ip == ip]
        comp = comps[0]
        lastPage = comp.sessions[-1].requests[-1].page
        actualPage = reqs[i].page
        indLast = PAGES.index(lastPage)
        indActual = PAGES.index(actualPage)
        timeLastStop = comp.sessions[-1].stopTime
        timeLastStart = comp.sessions[-1].startTime
        if (time - timeLastStop > th1) or (time - timeLastStart > th2) or not ((actualPage == lastPage) or (indActual in LINKS[indLast])):
            comp.sessions.append(Session(time, reqs[i]))
        else:
            comp.sessions[-1].requests.append(reqs[i])
            comp.sessions[-1].stopTime = time
            
sessionsCount = 0
for comp in computers:
    for s in comp.sessions:
        sessionsCount += 1
print("1) Liczba sesji: ", sessionsCount)

usersCount = 0
for comp in computers:
    usersCount += 1
print("2) Użytkowników: ", usersCount)

print("3) Sesji na jednego użytkownika: ", sessionsCount/usersCount)

lengths = [s.stopTime - s.startTime for comp in computers for s in comp.sessions]
lenPlot = plt.figure(1)
plt.hist(lengths, 50, alpha=0.5)
plt.title('4) Długości sesji (sekundy)')
lenPlot.show()

lengths = [len(s.requests) for comp in computers for s in comp.sessions]
lenPlot2 = plt.figure(2)
plt.hist(lengths, 50, alpha=0.5)
plt.title('5) Długości sesji (requests)')
lenPlot2.show()

times = [s.startTime/3600 for comp in computers for s in comp.sessions]
timesPlot = plt.figure(3)
beans = [i for i in range(25)]
plt.hist(times, beans, alpha=0.5)
plt.title('6) Godziny rozpoczęć sesji')
timesPlot.show()

entries = [0, 0, 0, 0, 0, 0]
departures = [0, 0, 0, 0, 0, 0]
timesOnPages = [0, 0, 0, 0, 0, 0]
pages = [0, 0, 0, 0, 0, 0]
for i in range(len(PAGES)):
    for comp in computers:
        for s in comp.sessions:
            if s.requests[0].page == PAGES[i]:
                entries[i] += 1
            elif s.requests[-1].page == PAGES[i]:
                departures[i] += 1
            for j in range(len(s.requests)):
                if s.requests[j].page == PAGES[i]:
                    pages[i] += 1
                    if not j == len(s.requests) - 1:
                        timesOnPages[i] += s.requests[j+1].time - s.requests[j].time
                    else:
                        nextInd = comp.sessions.index(s) + 1
                        if not nextInd == len(comp.sessions):
                            timesOnPages[i] += comp.sessions[nextInd].startTime - s.stopTime
                
entriesPlot = plt.figure(4)
x_axis = np.arange(len(PAGES))
plt.bar(x_axis, entries, align='center', alpha=0.5)
plt.xticks(x_axis, PAGES, rotation=90)
plt.title('7) Rozpoczęcia sesji na stronach')
entriesPlot.show()

departuresPlot = plt.figure(5)
x_axis = np.arange(len(PAGES))
plt.bar(x_axis, departures, align='center', alpha=0.5)
plt.xticks(x_axis, PAGES, rotation=90)
plt.title('8) Zakończenia sesji na stronach')
departuresPlot.show()

timeOnPagePlot = plt.figure(6)
x_axis = np.arange(len(PAGES))
avgTime = [(timesOnPages[i]/pages[i]) for i in range(6)]
plt.bar(x_axis, avgTime, align='center', alpha=0.5)
plt.xticks(x_axis, PAGES, rotation=90)
plt.title('9) Średnie czasy spędzone na stronach (s)')
timeOnPagePlot.show()

print('\n10) ')
nr = 1
presented = 10
for comp in computers:
    for s in comp.sessions:
        print(' ')
        if nr > presented: break
        print('===== Sesja nr ', nr)
        for r in s.requests:
            if not s.requests.index(r) == len(s.requests) - 1:
                print(r.page, end=' --> ')
            else:
                print(r.page)
                print(' ')
        nr +=  1
    if nr > presented:  break
        

Liczba logów:  2559
1) Liczba sesji:  1000
2) Użytkowników:  391
3) Sesji na jednego użytkownika:  2.557544757033248
